In [23]:
import os, sys
import ast
from datetime import date

from netCDF4 import Dataset
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import contextily as ctx
from pyproj import Transformer
import plotly.express as px

from bokeh.io import output_file, output_notebook, show
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, LogColorMapper, BasicTicker, ColorBar,
    DataRange1d, Range1d, PanTool, WheelZoomTool, BoxSelectTool, ResetTool, HoverTool
)
from bokeh.models.mappers import ColorMapper, LinearColorMapper
from bokeh.palettes import Viridis256

sys.path.append(os.path.abspath(os.path.join('..')))
from lib import utils

In [21]:
list(range(0, 24, 6))

[0, 6, 12, 18]

In [ ]:
lakes_gdf = utils.get_hrrrx_lake_output(date(2020, 4, 20), date(2020, 6, 20), cycle_hours=list(range(0, 24, 6)), pred_hours=[0])

In [6]:
# using manually-created bounding box for seneca lake
coords = [
    (-76.998076, 42.877521), 
    (-76.860185, 42.881791), 
    (-76.823285, 42.375953), 
    (-76.968944, 42.367344)
]

seneca_lake_bounds = gpd.GeoSeries([Polygon(coords)], crs='epsg:4326')
seneca_lake_bounds = seneca_lake_bounds.unary_union

In [7]:
lakes_gdf = lakes_gdf.assign(**{ 'seneca_lake': lakes_gdf.within(seneca_lake_bounds) })
seneca_lake_points = lakes_gdf[lakes_gdf['seneca_lake']]
seneca_lake_points.head()

,cycle_hour,fcst_datetime,pred_hour,pred_datetime,idx,depth,water_temp,air_temp,geometry,seneca_lake
3287,0,2020-04-20,0,2020-04-20,"[739, 1459]",10.0,NaN,NaN,POINT (-76.89417 42.46478),True
3309,0,2020-04-20,0,2020-04-20,"[742, 1458]",10.0,NaN,NaN,POINT (-76.90546 42.54949),True
3316,0,2020-04-20,0,2020-04-20,"[743, 1458]",10.0,NaN,NaN,POINT (-76.89734 42.57573),True
3327,0,2020-04-20,0,2020-04-20,"[744, 1458]",10.0,NaN,NaN,POINT (-76.88922 42.60197),True
3340,0,2020-04-20,0,2020-04-20,"[745, 1458]",10.0,NaN,NaN,POINT (-76.88110 42.62821),True


In [14]:
fig = px.line(
    seneca_lake_points,
    x='fcst_datetime',
    y='water_temp',
    color='idx',
    width=1920, 
    height=1080
)
fig.update_layout(
    title="HRRRX/CLM Lake Model 0hr-Preds for Seneca Lake, NY from Apr 20, 2020 - Jun 20, 2020 Forecast Cycles 00, 06, 12, and 18",
    xaxis_title="Forcast Cycle",
    yaxis_title="Water Temp (C)",
    xaxis = {
        'dtick': 3600000.0*24#*7
    }
)
fig.update_traces(connectgaps=False)
fig.show()